In [1]:
import folium
import pandas as pd
import numpy as np

In [2]:


folium.__version__

'0.2.1'

In [3]:
# Loading the P3 grant data

swiss_grant = r'P3_GrantExport.csv'
grant_data = pd.read_csv(swiss_grant,  sep=';', header=0, index_col=0, usecols=[0,6,7,13])

#Parsing a little the geographical data we have to allow for a simpler processing. Keep the rows where the canton is clearly indicated
#in the University column in the original dataframe, moving the others to a new dataframe _missing
grant_data_missing = grant_data[grant_data.University.isnull() | grant_data.University.isin(["Nicht zuteilbar - NA"])]
grant_data = grant_data.drop(grant_data_missing.index)

#dropping the rows for which we won't be able to extract the canton of funding
grant_data_missing = grant_data_missing[((grant_data_missing.University.isin(["Nicht zuteilbar - NA"]) | grant_data_missing.University.isnull() )& grant_data_missing.Institution.isnull()  ) == False]
#dropping rows where grant amount is not specified, since it cannot enrich our analysis
grant_data_missing.drop(grant_data_missing.ix[grant_data_missing['Approved Amount'] == "data not included in P3"].index, inplace= True)

grant_data['Canton']= grant_data["University"].str.split("- ").str[1]


print(grant_data.Institution.unique().size)
#We have a total of 5211 Institutions registered in the data. We aim for 95% geolocalisation, that is we shouldn't have more than 261 unparsed ones.

grant_data.loc[grant_data.Canton.isin(['LA', 'EPFL', 'HEPL', 'EHB', 'FORS', 'IST', 'SIB' ]), 'Canton'] = 'VD'
grant_data.loc[grant_data.Canton.isin(['PHBern', 'RWS', 'BFH']), 'Canton'] = 'BE'
grant_data.loc[grant_data.Canton.isin(['ETHZ', 'PHZFH', 'SIK-ISEA', 'EAWAG', 'EMPA', 'FHKD', 'HfH', 'ZFH']), 'Canton'] = 'ZH'
grant_data.loc[grant_data.Canton.isin(['IHEID']), 'Canton'] = 'GE'
grant_data.loc[grant_data.Canton.isin(['PHGR', 'PMOD', 'AORI', "und Asthmaforschung ", 'IKG']), 'Canton'] = 'GR'
grant_data.loc[grant_data.Canton.isin(['SUPSI', 'IRSOL', 'FUS', 'EOC', 'USI']), 'Canton'] = 'TI'
grant_data.loc[grant_data.Canton.isin(['PSI', 'PHFHNW', 'FIBL','FHNW']), 'Canton'] = 'AG'
grant_data.loc[grant_data.Canton.isin(['PHSG', 'SHLR', 'KSPSG', 'FHO']), 'Canton'] = 'SG'
grant_data.loc[grant_data.Canton.isin(['CSEM', 'ISSKA']), 'Canton'] = 'NE'
grant_data.loc[grant_data.Canton.isin(['HEPFR']), 'Canton'] = 'FR'
grant_data.loc[grant_data.Canton.isin(['FFHS', 'PHVS', 'CREALP', 'IDIAP', 'IUKB', 'IRO']), 'Canton'] = 'VS'
grant_data.loc[grant_data.Canton.isin(['STHB', 'FMI']), 'Canton'] = 'BS'
grant_data.loc[grant_data.Canton.isin(['PHSH']), 'Canton'] = 'SH'
grant_data.loc[grant_data.Canton.isin(['SPF', 'PHLU', 'HSLU']), 'Canton'] = 'LU'
grant_data.loc[grant_data.Canton.isin(['PHZG']), 'Canton'] = 'ZG'
grant_data.loc[grant_data.Canton.isin(['BITG', 'PHTG']), 'Canton'] = 'TG'
grant_data.loc[grant_data.Canton.isin(['PHSZ']), 'Canton'] = 'SZ'
grant_data.loc[grant_data.Canton.isin(['HEPBEJUNE']), 'Canton'] = 'JU'

grant_data_refine =grant_data[grant_data.Canton.isin(['NPO', 'HES-SO','ASPIT','FINST','WSL','FP','FTL','IST','ASP', 'AGS'])]
grant_data.drop(grant_data.ix[grant_data.Canton.isin(['NPO','HES-SO','ASPIT','FINST','WSL','FP','FTL','ASP', 'AGS'])].index, inplace= True)
#NPO, HES-SO, ASPIT, FINST,WSL, FP, FTL, ASP, AGS need further parsing
#ISO in Roma -> delete

grant_data_refine = pd.concat([grant_data_refine,grant_data_missing])
grant_data_refine.count()

5211


Approved Amount    4262
Canton             2722
Institution        4161
University         2903
dtype: int64

In [4]:
#Now, these are the 4342 rows we have left to parse, approximately 10% of the initial data. We aim for 95% parsed institutions.

grant_data_refine.loc[grant_data_refine.Institution.str.contains('Genève') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Geneva') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('travail social', case=0) == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('HEAD') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Droz') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CERN') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Martin Bodmer') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Muséum d'Histoire Naturelle") == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('genevoise') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Basel') == True, 'Canton'] = 'BS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Valais') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CREPA') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Sion') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Fribourg') == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Ecole d'ingénieurs et d'architectes") == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Forschungskomitee Rechtssoziologie") == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('fribourgeoise') == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Zür') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Eawag') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('ART') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Winterthur') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('ETH') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Arc', case=0) == True, 'Canton'] = 'JU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schaffhausen', case=0) == True, 'Canton'] = 'SH'

grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bibliothèque cantonale jurassienne', case=0) == True, 'Canton'] = 'JU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lausanne') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bibliothèque cantonale') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Vaud', case=0) == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('EPFL') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('EPF-L') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Gonin') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Suisse Occidental') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lullier') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('La Source') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Changins') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Ecole d'études sociales et pédagogiques") == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('HECV') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Heig-VD', case=0) == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CHUV') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bern') == True, 'Canton'] = 'BE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schweizerische Musikforschende Gesellschaft') == True, 'Canton'] = 'BE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Neuchâtel') == True, 'Canton'] = 'NE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Scherrer') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Hürlimann') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Baden') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Aar') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('cantonale', case=0) == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Ticin', case=0) == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lugano') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Liceo') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Svizzera') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Sezione') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Luzern') == True, 'Canton'] = 'LU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Thurgau') == True, 'Canton'] = 'TG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('SLF') == True, 'Canton'] = 'GR'  
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Rumants') == True, 'Canton'] = 'GR' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schwyz') == True, 'Canton'] = 'SZ'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('St. Gall') == True, 'Canton'] = 'SG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('St.Gall') == True, 'Canton'] = 'SG' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Zug') == True, 'Canton'] = 'ZG' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Solothurn') == True, 'Canton'] = 'SO' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Appenzell') == True, 'Canton'] = 'AR' 

grant_data_refined = grant_data_refine[grant_data_refine.Canton.isin(['GE','SZ','SO','AG', 'TI', 'ZG', 'GR','SG', 'BS','VS', 'LU', 'FR','ZH', 'LA', 'BE', 'NE', 'AR', 'JU', 'TG'])]
grant_data_refine.drop(grant_data_refine.ix[grant_data_refine.Canton.isin(['ZG','SZ','SO','AG', 'TI', 'GE','GR','SG', 'BS','LU', 'TG', 'VS','FR','ZH', 'LA', 'BE', 'NE', 'AR', 'JU'])].index, inplace= True)

#grant_data_refine.loc[grant_data_refine.University.isnull()].Institution.unique().size / grant_data_refine.loc[grant_data_refine.University.isnull()].Institution.size
#We have 95% of unique Insitution in the list, most of them out of Switzerland, so it is safe to ditch everything left here.
grant_data_refine.drop(grant_data_refine.ix[grant_data_refine.University.isnull()].index, inplace= True)


#pursuing the parsing...
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Institute', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Department', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('University', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Universität', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('UNI', case=0) == True].index, inplace= True)





#What we use to identify the non parsed institutions
#grant_data_refine.sort_values('Approved Amount', ascending=False).Institution.unique()
#grant_data_refine.Institution.size

grant_data = pd.concat([grant_data,grant_data_refined])
grant_data.dropna(subset=['Canton'],inplace=True)
grant_data.drop(grant_data.ix[grant_data.Canton.isin(['NPO','HES-SO','ASPIT','FINST','WSL','FP','FTL','ASP', 'AGS', 'ISR'])].index, inplace= True)


In [5]:
#Finally, computing the sums
grant_data['Approved Amount'] = pd.to_numeric(grant_data['Approved Amount'], errors='coerce')
grant_canton = grant_data.groupby(grant_data.Canton).sum()
grant_canton.loc['UR'] = 0.0
grant_canton.loc['NW'] = 0.0
grant_canton.loc['OW'] = 0.0
grant_canton.loc['GL'] = 0.0
grant_canton.loc['BL'] = 0.0
grant_canton.loc['AI'] = 0.0
grant_canton['cantons'] = grant_canton.index
grant_canton.sort_values('Approved Amount', ascending = False)

,Approved Amount,cantons
Canton,,
ZH,3.695114e+09,ZH
VD,2.415326e+09,VD
GE,1.901525e+09,GE
BE,1.581336e+09,BE
BS,1.398939e+09,BS
FR,4.614244e+08,FR
NE,4.050351e+08,NE
AG,1.798424e+08,AG
TI,1.247166e+08,TI


In [6]:
#Build the folium map of Switzerland with highlighted cantons

map = folium.Map(location=[46.8,8],  tiles='Mapbox Bright', zoom_start=8)
swiss_topo = r'ch-cantons.topojson.json'

map.choropleth(geo_path = swiss_topo, topojson = 'objects.cantons',
               data=grant_canton,
               columns=['cantons', 'Approved Amount'],
               key_on='feature.id',
               fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
               legend_name='Grant Amount',
               threshold_scale = [ 1.7e+05, 3.6e+07, 1.0e+08, 1.3e+09, 2.4e+09, 3.69e+09],
               reset=True)
map

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 905: ordinal not in range(128)

## Bonus


We divide the problem by looking at the cantons
[image](https://fr.wikipedia.org/wiki/R%C3%B6stigraben#/media/File:Sprachen_CH_2000_fr.png)

**French speaking Universities**

In [7]:
french_university = grant_data[grant_data.Canton.isin(['GE','JU', 'VD','NE'])]
french_university = french_university.copy()
french_university['Language']='FR'

**German speaking Universities**

In [8]:
german_university = grant_data[grant_data.Canton.isin(['SZ','SO','AG', 'ZG', 'GR','SG', 'BS', 'LU','ZH', 'LA', 'BE', 'AR', 'TG'])]
german_university = german_university.copy()
german_university['Language']='DE'

### Other Universities 

#### Where to cut the Rostigraben ?
Valais, Fribourg and Bern are the 3 cantons that are cut by the Rostigraben. However, depending on where the university is in the canton, it might be in the french or german speaking part.
Let's have a look at each of these university for each canton. We will look for the name of the university on google, and depending on the language spoken in the university's city, we will assign a language.

#### Valais 
In valais, all the univeristy are in the French speaking part, besides :

- 'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS'
- 'Pädagogische Hochschule Wallis - PHVS'

which are located in Brig, in the Swiss German part

In [9]:
a = grant_data[grant_data.Canton.isin(['VS'])]
a['University'].unique()

array(["Centre de rech. sur l'environnement alpin - CREALP",
       'Idiap Research Institute - IDIAP',
       'Institut Universitaire Kurt Bösch - IUKB',
       'Forschungsinstitut für Opthalmologie - IRO',
       'Pädagogische Hochschule Wallis - PHVS',
       'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Weitere Spitäler - ASPIT', 'HES de Suisse occidentale - HES-SO'], dtype=object)

In [10]:
valais_university = grant_data[grant_data.Canton.isin(['VS'])]
valais_university = valais_university.copy()
valais_university['Language'] = 'FR'
valais_university.loc['Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS','Language'] = 'DE'
valais_university.loc['Pädagogische Hochschule Wallis - PHVS','Language'] = 'DE'

#### Fribourg

**Assumption** We say that the city of Fribourg is french speaking as 80% of the population speak French


In Fribourg, all the univeristy are in the city of Fribourg ( French speaking) , but :

- 'Firmen/Privatwirtschaft - FP'

which is located in Fribourg, but speaks German.

In [11]:
a = grant_data[grant_data.Canton.isin(['FR'])]
a['University'].unique()

array(['Université de Fribourg - FR',
       'Haute école pédagogique fribourgeoise - HEPFR',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Firmen/Privatwirtschaft - FP',
       'HES de Suisse occidentale - HES-SO', nan], dtype=object)

In [12]:
fribourg_university = grant_data[grant_data.Canton.isin(['FR'])]
fribourg_university = fribourg_university.copy()
fribourg_university['Language'] = 'FR'
fribourg_university.loc['Firmen/Privatwirtschaft - FP'] = 'DE'

#### Bern

**Assumption** We say that the city of Bern is German speaking.


In Bern, all the univeristies are in the city of Bern ( German speaking)..

In [13]:
a = grant_data[grant_data.Canton.isin(['BE'])]
a['University'].unique()

array(['Universität Bern - BE', 'Robert Walser-Stiftung Bern - RWS',
       'Berner Fachhochschule - BFH',
       'Pädagogische Hochschule Bern - PHBern',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Firmen/Privatwirtschaft - FP',
       'Forschungsanstalten Agroscope - AGS', 'Weitere Institute - FINST',
       nan], dtype=object)

In [14]:
bern_university = grant_data[grant_data.Canton.isin(['BE'])]
bern_university = bern_university.copy()
bern_university['Language'] = 'DE'

#### Concatenate all the dataFrames

In [15]:
language_cantons = pd.concat([french_university,german_university,valais_university,fribourg_university,bern_university])

In [37]:
#Finally, computing the sums
language_cantons['Approved Amount'] = pd.to_numeric(language_cantons['Approved Amount'], errors='coerce')
language_cantons_ = language_cantons.groupby(language_cantons.Language).sum()
language_cantons1 = language_cantons.groupby(language_cantons.Language).count()

language_cantons_.sort_values('Approved Amount', ascending = False)



,Approved Amount
Language,
DE,8.658452e+09
FR,5.272519e+09


Counting the number of university in each part of Rostigraben

In [35]:
df=language_cantons[['University','Language']]
df=df.drop_duplicates()
df.groupby(df.Language).count()


,University
Language,
DE,46
FR,29
